In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Step to calibrate camera


In [21]:
def get_chess_image_point_corner_pts(num_row, num_col, glob_path):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((num_col*num_row,3), np.float32)
    objp[:,:2] = np.mgrid[0:num_row, 0:num_col].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob(glob_path)
    print ("found " + str(len(images)) + " image")
    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        print ("Processing " + fname, end = ', ')
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (num_row, num_col), None)

        # If found, add object points, image points
        if ret == True:
            print ("found corner:" + str(num_row) + "x" + str(num_col))
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (num_row ,num_col), corners, ret)
            cv2.imshow('img', img)
            cv2.waitKey(500)

    cv2.destroyAllWindows()    
    
#def calibate_camera(glob_path, ):

In [22]:
glob_path = 'camera_cal/calibration*.jpg'

In [23]:
get_chess_image_point_corner_pts(9, 6, glob_path)

found 20 image
Processing camera_cal/calibration1.jpg, Processing camera_cal/calibration10.jpg, found corner:9x6
Processing camera_cal/calibration11.jpg, found corner:9x6
Processing camera_cal/calibration12.jpg, found corner:9x6
Processing camera_cal/calibration13.jpg, found corner:9x6
Processing camera_cal/calibration14.jpg, found corner:9x6
Processing camera_cal/calibration15.jpg, found corner:9x6
Processing camera_cal/calibration16.jpg, found corner:9x6
Processing camera_cal/calibration17.jpg, found corner:9x6
Processing camera_cal/calibration18.jpg, found corner:9x6
Processing camera_cal/calibration19.jpg, found corner:9x6
Processing camera_cal/calibration2.jpg, found corner:9x6
Processing camera_cal/calibration20.jpg, found corner:9x6
Processing camera_cal/calibration3.jpg, found corner:9x6
Processing camera_cal/calibration4.jpg, Processing camera_cal/calibration5.jpg, Processing camera_cal/calibration6.jpg, found corner:9x6
Processing camera_cal/calibration7.jpg, found corner:9x6